In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,                # We will only detect one hand
    min_detection_confidence=0.7,   # Minimum confidence for hand detection
    min_tracking_confidence=0.5     # Minimum confidence for tracking
)
mp_draw = mp.solutions.drawing_utils

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)

# Define the landmark IDs for the fingertips
tip_ids = [4, 8, 12, 16, 20]

print("Starting camera feed. Press 'q' to quit.")

while True:
    # 1. Read a frame from the webcam
    success, image = cap.read()
    if not success:
        print("Failed to capture image from camera.")
        break

    # Flip the image horizontally for a later selfie-view display
    # and convert the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    
    # 2. Process the image and find hands
    results = hands.process(image)

    # Mark the image as writeable again to draw on it
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    landmark_list = []
    
    # 3. If a hand is detected, get its landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the coordinates of all landmarks
            for id, lm in enumerate(hand_landmarks.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmark_list.append([id, cx, cy])

            # Draw the landmarks and connections on the image
            mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # 4. Count the number of extended fingers
    finger_count = 0
    if len(landmark_list) != 0:
        fingers = []

        # Thumb: Check if the thumb tip is to the left (for right hand) 
        # or right (for left hand) of the point below it.
        # This logic works for a flipped image.
        if landmark_list[tip_ids[0]][1] > landmark_list[tip_ids[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Four Fingers: Check if the tip of the finger is above the joint below it
        for id in range(1, 5):
            if landmark_list[tip_ids[id]][2] < landmark_list[tip_ids[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        
        finger_count = fingers.count(1)

    # 5. Display the finger count on the image
    cv2.putText(
        image, 
        f"Fingers: {finger_count}", 
        (50, 50), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (255, 0, 0), 
        2, 
        cv2.LINE_AA
    )

    # 6. Show the image in a window
    cv2.imshow("Finger Counter", image)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# 7. Release resources
cap.release()
cv2.destroyAllWindows()
# This is important for Jupyter to properly close the window
cv2.waitKey(1) 
print("Camera feed stopped.")



I0000 00:00:1761254269.420027  551356 gl_context.cc:357] GL version: 2.1 (2.1 ATI-7.0.23), renderer: AMD Radeon Pro 5500M OpenGL Engine
W0000 00:00:1761254269.444445  553791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761254269.491653  553791 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-10-24 02:47:49.923 python[40007:551356] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Starting camera feed. Press 'q' to quit.


/opt/anaconda3/envs/open-cv-ml/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

: 